In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
actions_files = """ 
name: Run tests on pull requests

on:
  pull_request:
    branches: [main]
  push:
    branches: [main]

jobs:
  run-tests:
    runs-on: ubuntu-latest
    name: Run test suite

    # https://github.com/marketplace/actions/setup-miniconda#use-a-default-shell
    defaults:
      run:
        shell: bash -l {0}

    steps:
      - name: Checkout repository
        uses: actions/checkout@v2

      # See: https://github.com/marketplace/actions/setup-miniconda
      - name: Setup miniconda
        uses: conda-incubator/setup-miniconda@v2
        with:
          auto-update-conda: true
          miniforge-variant: Mambaforge
          channels: conda-forge
          activate-environment: llamabot
          environment-file: environment.yml
          use-mamba: true

      - name: Run tests
        run: |
          conda activate llamabot
          pip install -e .
          pytest

      # https://github.com/codecov/codecov-action
      - name: Upload code coverage
        uses: codecov/codecov-action@v2
        with:
          # fail_ci_if_error: true # optional (default = false)
          verbose: true # optional (default = false)
"""

In [ ]:
from llamabot import SimpleBot

bot = SimpleBot(
    "You are an expert CI/CD engineer who knows how to write GitHub Actions workflows."
)

In [ ]:
from outlines import text


@text.prompt
def modify_workflow_file(workflow_file: str, request: str) -> str:
    """Please modify the workflow file with the following request.

    workflow file:

    {{ workflow_file }}

    request:

    {{ request }}
    """

In [ ]:
bot(
    modify_workflow_file(
        actions_files,
        "Make this matrixed: I want to test installation of dependencies from both `environment.yml` and within a bare Python 3.10 environment.",
    )
)

In [ ]:
code1 = ''' 
@app.command()
def code_generator(request: str):
    """Generate code.

    :param request: A description of what the code should do.
    """
    accepted = False
    while not accepted:
        code = codebot(ghostwriter(request, "Python"))
        user_response = get_valid_input("Do you accept this code? (y/n) ")
        if user_response == "y":
            accepted = True
            break
        elif user_response == "n":
            continue
    if accepted:
        pyperclip.copy(code.content)
        print("Copied to clipboard!")
'''

code2 = ''' 
@app.command()
def function_docstrings(module_fpath: Path, function_name: str, style: str = "sphinx"):
    """Generate function docstrings.

    :param module_fpath: Path to the module to generate docstrings for.
    :param function_name: Name of the function to generate docstrings for.
    :param style: Style of docstring to generate.
    """
    accepted = False
    while not accepted:
        # source_code = read_file(module_fpath)
        function_source = get_object_source_code(module_fpath, function_name)
        function_docstring = codebot(docstring(function_source, style=style))
        print("\n\n")
        user_response = get_valid_input("Do you accept this docstring? (y/n) ")
        if user_response == "y":
            accepted = True
            break
        elif user_response == "n":
            continue
    if accepted:
        pyperclip.copy(function_docstring.content)
        print("Copied to clipboard!")
'''

code3 = ''' 

@app.command()
def module_docstrings(module_fpath: Path):
    """Generate module-level docstrings.

    :param module_fpath: Path to the module to generate docstrings for.
    """
    accepted = False
    while not accepted:
        source_code = read_file(module_fpath)
        module_docstring = codebot(module_doc(source_code))
        print("\n\n")
        user_response = get_valid_input("Do you accept this docstring? (y/n) ")
        if user_response == "y":
            accepted = True
            break
        elif user_response == "n":
            continue
    if accepted:
        pyperclip.copy(module_docstring.content)
        print("Copied to clipboard!")
'''

In [ ]:
from outlines import text


@text.prompt
def refactor_similarities(code1, code2, code3):
    """I have two code blocks that are very similar.


    code1:

    {{ code1 }}

    code2:

    {{ code2 }}

    code3:

    {{ code3 }}

    How can I refactor them to reduce code duplication?
    """


from llamabot.prompt_library.coding import codebot

codebot(refactor_similarities(code1, code2, code3))